# 0 Imports

In [37]:
import numpy as np
import matplotlib as pl
import seaborn as sns
import pandas as pd

from bs4 import BeautifulSoup as bs
import spacy
nlp = spacy.load('en_core_web_sm')

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

np.random.seed(0)


***
# 1 Config

In [29]:
config = {
    "raw_preproc": True
}

***
# 2 Raw data preprocessing

## 2.1 Parse raw data body html

In [26]:
if config["raw_preproc"]:
    raw_data_body = pd.read_csv("data/raw_data_body.csv")
    raw_data_body.shape

In [30]:
if config["raw_preproc"]:
    def preproc_raw_data_body(cell):
        preproc_cell = bs(cell, "html.parser").get_text()
        preproc_cell = preproc_cell.replace(',', ' ')
        return preproc_cell

In [31]:
if config["raw_preproc"]:
    raw_data_body["Body"] = raw_data_body.apply(lambda row: preproc_raw_data_body(row.Body), axis=1)

***
## 2.2 Merge body raw data to the rest

In [38]:
raw_data_rest = pd.read_csv("data/raw_data_id_title_tags.csv")

In [70]:
raw_data = raw_data_rest.join(raw_data_body)
raw_data = raw_data.set_index("Id").reindex(["Title", "Body", "Tags"], axis="columns")

In [72]:
raw_data.head(100)

,Title,Body,Tags
Id,,,
4,How to convert a Decimal to a Double in C#?,I want to use a Track-Bar to change a Form's o...,<c#><floating-point><type-conversion><double><...
6,Why did the width collapse in the percentage w...,I have an absolutely positioned div containing...,<html><css><internet-explorer-7>
7,NaN,An explicit cast to double like this isn't nec...,NaN
9,How do I calculate someone's age based on a Da...,Given a DateTime representing a person's birth...,<c#><.net><datetime>
11,Calculate relative time in C#,Given a specific DateTime value how do I disp...,<c#><datetime><time><datediff><relative-time-s...
12,NaN,Here's how I do it\nvar ts = new TimeSpan(Date...,NaN
13,Determine a user's timezone,Is there a standard way for a web server to be...,<html><browser><timezone><user-agent><timezone...
14,Difference between Math.Floor() and Math.Trunc...,What is the difference between Math.Floor() an...,<.net><math>
16,Filling a DataSet or a DataTable from a LINQ q...,How do you expose a LINQ query as an ASMX web ...,<c#><linq><web-services><.net-3.5>


***
## 2.x Raw data loading